<a href="https://colab.research.google.com/github/Rusira54321/RAG-APPLICATION-Medium-level-/blob/main/ragapplication1234.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU
!pip install langchain_community -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 19.8 MB/s et

In [3]:
import os
from google.colab import userdata

In [4]:
from langchain_openai import ChatOpenAI
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 10.1 MB/s eta 0:00:00


In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/codeprolk.pdf")
docs = loader.load()
print(docs)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-18T10:00:42+05:30', 'author': 'Dinesh Piyasamara', 'moddate': '2024-06-18T10:00:42+05:30', 'source': '/content/codeprolk.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content="Introduction to CodePRO LK \nCodePRO LK is a dynamic educational platform that offers a diverse range of technology-\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara during the \nCOVID-19 pandemic, CodePRO LK addresses the growing need for accessible, high-quality \ntech education tailored to the local community. \n \nFounding and Vision \nOrigin and Motivation \nThe inception of CodePRO LK was driven by the challenges posed by the COVID-19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dines

In [9]:
len(docs)

4

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
splites = text_splitter.split_documents(docs)

In [11]:
len(splites)

20

In [12]:
print(splites)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-18T10:00:42+05:30', 'author': 'Dinesh Piyasamara', 'moddate': '2024-06-18T10:00:42+05:30', 'source': '/content/codeprolk.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Introduction to CodePRO LK \nCodePRO LK is a dynamic educational platform that offers a diverse range of technology-\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara during the \nCOVID-19 pandemic, CodePRO LK addresses the growing need for accessible, high-quality'), Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-18T10:00:42+05:30', 'author': 'Dinesh Piyasamara', 'moddate': '2024-06-18T10:00:42+05:30', 'source': '/content/codeprolk.pdf', 'total_pages': 4

In [13]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=splites,embedding=embedding_model)

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
     "You are an intelligent chatbot. Use the following context to answer the question. If you don't know  the answer in wonderful way, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{question}"),
    ]
)

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know  the answer in wonderful way, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [17]:
from langchain_core.runnables import RunnablePassthrough
rag_Chain  = (
    {
        "context":retriever,
        "question":RunnablePassthrough(),
    }
    |
    prompt
    |
    llm
)

In [18]:
response = rag_Chain.invoke("CAn you list down")
response.content

"I'm sorry, but I don't know what you would like me to list down. Could you please provide more details?"

In [19]:
response = retriever.invoke("can you list down")
response

[Document(id='ecadca4b-f74d-43e3-90b8-0f456b430cc7', metadata={'page': 3, 'producer': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-18T10:00:42+05:30', 'page_label': '4', 'total_pages': 4, 'source': '/content/codeprolk.pdf', 'moddate': '2024-06-18T10:00:42+05:30', 'creator': 'Microsoft® Word for Microsoft 365', 'author': 'Dinesh Piyasamara'}, page_content='Community Engagement and Events \nCodePRO LK is committed to strengthening its community through regular engagement \nactivities such as webinars, live coding sessions, hackathons, and tech talks. These events \nprovide valuable networking opportunities and practical experience, fostering a supportive and \ncollaborative learning environment. \n \nConclusion'),
 Document(id='0d71acc9-dec2-40c1-b69a-9d36f1fcb3e1', metadata={'creationdate': '2024-06-18T10:00:42+05:30', 'source': '/content/codeprolk.pdf', 'author': 'Dinesh Piyasamara', 'producer': 'Microsoft® Word for Microsoft 365', 'moddate': '2024-06-18T10:00:42+05:30

In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know  the answer in wonderful way, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system","using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"),
    MessagesPlaceholder("chat_history"),
    ("human","{input}"),
])

# Create the history aware retriever
history_aware_retriever = (
    rewrite_prompt
    |
    llm
    |
    (lambda x:x.content)
    |
    retriever
)

In [25]:
from langchain_core.prompts import MessagesPlaceholder
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know in wonderful way"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [42]:
from langchain_core.runnables import RunnableLambda
rag_chain = (
    {
        "context": history_aware_retriever,
        "input": RunnablePassthrough(),
        "chat_history": RunnableLambda(lambda x: x["chat_history"]),
    }
    | prompt
    | llm
)

In [43]:
from langchain_core.messages import HumanMessage,AIMessage
chat_history = []

In [47]:
question = "can you describe it briefly"
response1 = rag_chain.invoke(
    {
        "input": question,
        "chat_history": chat_history
    }
)

print(response1.content)

chat_history.append(HumanMessage(content=question))
chat_history.append(AIMessage(content=response1.content))

CodePRO LK is an educational platform that offers technology courses in Sinhala, founded by Dinesh Piyasamara during the COVID-19 pandemic. Its goal is to empower Sri Lankans with skills in programming, data science, and machine learning, making high-quality tech education accessible to the local community.


In [48]:
chat_history

[HumanMessage(content='Who is CodeProLk?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in programming, data science, and machine learning. The platform focuses on providing accessible, high-quality tech education tailored to the local community.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Who is CodeProLk?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='CodePRO LK is an educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It provides a variety of technology-related courses in Sinhala, focusing on empowering Sri Lankans with skills in programming, data science, and machine learning. The platform aims to make high-quality tech education accessible to the local community.', ad

[HumanMessage(content='Who is CodeProLk?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in programming, data science, and machine learning. The platform focuses on providing accessible, high-quality tech education tailored to the local community.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Who is CodeProLk?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It seems you're looking for a list of information about CodePRO LK. Here’s a summary of key points:\n\n1. **Founder**: Dinesh Piyasamara\n2. **Established**: During the COVID-19 pandemic\n3. **Focus**: Technology-related education\n4. **Language**: Courses are offered in Sinhala\n5. **Target Audience**: Aimed at empowering Sri Lankans\n6. **Skills Off